In [ ]:
from uuid import uuid4
import nest_asyncio
import asyncio
import aiohttp
import time

# 주피터 노트북에서 사용하려면 이렇게 해야 함.
try:
    nest_asyncio.apply()
except RuntimeError:
    pass


async def make_request(session, url, id):
    try:
        async with session.post(url, json={"id": id}) as response:
            return await response.json()
    except Exception as e:
        print(f"Request failed: {e}")
        return None


async def stress_test(url, num_requests):
    async with aiohttp.ClientSession() as session:
        tasks = [make_request(session, url, str(uuid4())) for _ in range(num_requests)]
        start_time = time.time()
        responses = await asyncio.gather(*tasks, return_exceptions=True)
        end_time = time.time()
    
    total_time = end_time - start_time
    requests_per_second = num_requests / total_time if total_time > 0 else 0

    success_count = sum(1 for r in responses if r is not None and r["code"] == 0)
    failure_count = num_requests - success_count

    print(f"Completed {num_requests} requests in {total_time:.2f} seconds")
    print(f"Requests per second: {requests_per_second:.2f}")
    print(f"Successful requests: {success_count}")
    print(f"Failed requests: {failure_count}")


if __name__ == '__main__':
    api_url = "http://localhost:8080/redisonly/coupon"  # Replace with your API endpoint
    num_requests = 100  # Adjust the number of requests as needed
    
    try:
        asyncio.run(stress_test(api_url, num_requests))
    except RuntimeError:
        # 주피터 노트북에서 `asyncio.run` 문제 해결
        loop = asyncio.get_event_loop()
        loop.run_until_complete(stress_test(api_url, num_requests))
        
"""
<결론>
동일 cpu, memory 기준에서는 단일 Redis 노드로 수행하는 것이 비슷하거나 약간 더 나았다.

===================================================================
# CPU 종류: 12th Gen Intel(R) Core(TM) i7-12700H
# 코어: 14개
# 논리 프로세서: 20개

# Redis node 1개 당: 1cpu, 128M mem

<Redis node 1개만 사용, 1000개의 requests, 딱 1회만 실행(캐싱 방지)>
# 선착순 쿠폰 10000개

spring boot: 1cpu, 1024M mem -> 40TPS, spring boot 실행시간 17초
spring boot: 1cpu, 2048M mem -> 70TPS, spring boot 실행시간 18초
spring boot: 8cpu, 1024M mem -> 440TPS, spring boot 실행시간 12초, 번외) 10000개 기준 1126TPS, 아예 다시 실행해봤더니 1000개 기준 547TPS..?
spring boot: 8cpu, 2048M mem -> 400TPS, spring boot 실행시간 11초
spring boot: 8cpu, 4096M mem -> 500TPS, spring boot 실행시간 13초
spring boot: 16cpu, 1024M mem -> 220TPS, spring boot 실행시간 12초
spring boot: 16cpu, 4096M mem -> 310TPS, spring boot 실행시간 12초
spring boot: 14cpu, 1024M mem -> 505TPS, spring boot 실행시간 13초
spring boot: 14cpu, 4096M mem -> 420TPS, spring boot 실행시간 12초


<Redis node 5개를 cluster로 사용, 1000개 || 10000개의 requests, 딱 1회만 실행(캐싱 방지)>
# 선착순 쿠폰 10000개
# lettuce pool을 5개로 고정해놓고 실험했음.
# 들어온 request에 랜덤하게 5개의 hash tag 중 하나를 붙이는 식으로 부하를 분산했음. 그래서 선착순 쿠폰이 남아 있어도 실패할 수도 있다.


spring boot: 8cpu, 1024M mem -> 440TPS || 1216TPS, spring boot 실행시간 13초, 51실패(10000개 중)
spring boot: 8cpu, 4096M mem -> 364TPS || 1135TPS, spring boot 실행시간 12초, 60실패(10000개 중)
spring boot: 14cpu, 1024M mem -> 466TPS || 921TPS, spring boot 실행시간 13초, 100실패(10000개 중)

"""